单任务测试

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

from uhpc.class_uhpc import DataSet_uhpc
tasks = {
            'Flowability': 21,
            'Porosity': 22,
            'Compressive strength': 24,
            'Flexural strength': 24
        }
for task, feature_num in tasks.items():
    if task == 'Flowability':
        data_model = DataSet_uhpc('./',feature_num, task_name = task)
        train_data, x_test_f, y_test_f = data_model.get_data()
        columns_to_drop = [task,22,23,24]
        X_train_f = train_data.drop(columns=columns_to_drop).reset_index(drop=True)
        y_train_f = train_data[[task]].reset_index(drop=True)
    elif task == 'Porosity':
        data_model = DataSet_uhpc('./',feature_num, task_name = task)
        train_data, x_test_p, y_test_p = data_model.get_data()
        columns_to_drop = [task,23,24]
        X_train_p = train_data.drop(columns=columns_to_drop).reset_index(drop=True)
        y_train_p = train_data[[task]].reset_index(drop=True)
    elif task == 'Compressive strength':
        data_model = DataSet_uhpc('./',feature_num, task_name = task)
        train_data, x_test_cs, y_test_cs = data_model.get_data()
        columns_to_drop = [task]
        X_train_cs = train_data.drop(columns=columns_to_drop).reset_index(drop=True)
        y_train_cs = train_data[[task]].reset_index(drop=True)
    elif task == 'Flexural strength':
        data_model = DataSet_uhpc('./',feature_num, task_name = task)
        train_data, x_test_fs, y_test_fs = data_model.get_data()
        columns_to_drop = [task]
        X_train_fs = train_data.drop(columns=columns_to_drop).reset_index(drop=True)
        y_train_fs = train_data[[task]].reset_index(drop=True)


多输出模式

In [1]:
import pandas as pd
# 读取数据并处理
multi_data = pd.read_excel('./UHPC.xlsx')
multi_data_ = multi_data.copy()

# 测试集：行内没有任何缺失
test_mask = multi_data_.notna().all(axis=1)
test_data = multi_data_.loc[test_mask].reset_index(drop=True)

# 训练集：行内存在任意缺失
train_mask = ~test_mask   # 等价于 multi_data_.isna().any(axis=1)
train_data = multi_data_.loc[train_mask].reset_index(drop=True)
X_train = train_data.iloc[:,:24]
y_train = train_data.iloc[:,24:]
X_test = test_data.iloc[:,:24]
y_test = test_data.iloc[:,24:]

In [3]:
from uhpc.class_method import ModelInter
model_inter = ModelInter(X_train, y_train, X_test, y_test)
pred = model_inter._impute()

CatBoost R2: 0.265, MSE: 27.323, MAE: 3.610
MT-ExtraTrees R2: -19.521, MSE: 290.730, MAE: 13.623


In [4]:
from uhpc.class_method import IterativeInter
inter = IterativeInter(X_train, y_train, X_test, y_test)
pred = inter.fit_predict()

XGBoostError: [00:48:48] C:\actions-runner\_work\xgboost\xgboost\src\data\data.cc:565: Check failed: valid: Label contains NaN, infinity or a value too large.

In [2]:
from uhpc.class_method import GPReg
gpreg = GPReg(X_train, y_train, X_test, y_test)
pred = gpreg._model()

RuntimeError: You must train on the training inputs!

In [3]:
from uhpc.class_method import int_multi
int_m = int_multi(X_train, y_train, X_test, y_test)
output = int_m.fit_pre()

Imputing method: missforest

Imputing method: hyperimpute

Imputing method: gain

Imputing method: sinkhorn

Imputing method: KNN

Imputing method: MIDA

Imputing method: miwae

Imputing method: gtmcc

Imputing method: lm

Imputing method: InterativeImputer



C:\Users\weiso\PycharmProjects\MTL_in_MAT\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\weiso\PycharmProjects\MTL_in_MAT\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\weiso\PycharmProjects\MTL_in_MAT\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MultiTaskLasso was fitted with feature names
  warnings.warn(
C:\Users\weiso\PycharmProjects\MTL_in_MAT\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MultiTaskElasticNet was fitted with feature names
  warnings.warn(
C:\Users\weiso\PycharmProjects\MTL_in_MAT\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

In [52]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
models = [
'RandomForestRegressor',
'Ridge',
'MultiTaskLasso',
'MultiTaskElasticNet',
'MultiOutputRegressor',
'MultiOutputRegressor'
]
methods = [
    'missforest',
    'hyperimpute',
    'gain',
    'sinkhorn',
    'KNN',
    'MIDA',
    'miwae',
    'gtmcc',
    'lm',
    'InterativeImputer'
]

idx = { (r['model'], r['method']): r['y_pred'] for r in output }
       # 只得到 y_pred
result = []
for method in methods:
    for model in models:
        y_pred = idx[(model, method)]
        print('='*50)
        print('Method:', method, 'Model:', model)
        print(y_pred.shape, y_test.shape)
        R2 = r2_score(y_test, y_pred, multioutput='raw_values')
        print('R2:', R2)
        R2_kom = r2_score(y_test, y_pred, multioutput='uniform_average')
        print('R2_kom:', R2_kom)
        RMSE = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
        print('RMSE:', RMSE)
        RMSE_kom = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='uniform_average'))
        print('RMSE_kom:', RMSE_kom)
        RRMSE = np.sqrt(np.abs(1 - R2))
        print('RRMSE:', RRMSE)
        RRMSE_kom = np.sqrt(np.abs(1 - R2_kom))
        print('RRMSE_kom:', RRMSE)
        result.append(
            {
        'model': model,
        'method': method,
        'R2_kom': R2_kom,
        'RMSE_kom': RMSE_kom,
        'RRMSE_kom': RRMSE_kom,
        'R2': R2,
        'RMSE': RMSE,
        'RRMSE': RRMSE
            }
        )

Method: missforest Model: RandomForestRegressor
(47, 4) (47, 4)
R2: [ 0.688662    0.59839483 -1.02288666  0.24194546]
R2_kom: 0.12652890561204574
RMSE: [10.25147765  3.28864823  4.6451441   3.03465135]
RMSE_kom: 6.055874702729029
RRMSE: [0.5579767  0.63372326 1.4222822  0.87066328]
RRMSE_kom: [0.5579767  0.63372326 1.4222822  0.87066328]
Method: missforest Model: Ridge
(47, 4) (47, 4)
R2: [ 0.07184775 -0.00230317 -1.15261243  0.25856763]
R2_kom: -0.2061250537539272
RMSE: [17.700275    5.19538007  4.79177412  3.00119592]
RMSE_kom: 9.647020073495671
RRMSE: [0.96340659 1.00115092 1.46717839 0.86106467]
RRMSE_kom: [0.96340659 1.00115092 1.46717839 0.86106467]
Method: missforest Model: MultiTaskLasso
(47, 4) (47, 4)
R2: [-0.10501294  0.01672663 -1.18941047  0.3495337 ]
R2_kom: -0.2320407728602409
RMSE: [19.31319319  5.1458238   4.83255729  2.81106558]
RMSE_kom: 10.377073689072196
RRMSE: [1.05119596 0.99160142 1.47966566 0.80651491]
RRMSE_kom: [1.05119596 0.99160142 1.47966566 0.80651491]
Me

In [57]:
df = pd.DataFrame(result)
metrics = {
    'R2_kom': False,
    'RMSE_kom': True,
    'RRMSE_kom': True,
}
top5 = {}
for m, asc in metrics.items():
    if m in df.columns:
        top5[m] = (
            df.sort_values(
                by = m, ascending = asc
            )
            .head(5)
            .loc[:, ['model', 'method', m]]
            .reset_index(drop=True)
        )

In [60]:
top5

{'R2_kom':                    model      method    R2_kom
 0  RandomForestRegressor    sinkhorn  0.354767
 1  RandomForestRegressor  missforest  0.126529
 2   MultiOutputRegressor    sinkhorn  0.048679
 3   MultiOutputRegressor    sinkhorn  0.048679
 4                  Ridge    sinkhorn  0.017618,
 'RMSE_kom':                    model       method  RMSE_kom
 0  RandomForestRegressor   missforest  6.055875
 1  RandomForestRegressor     sinkhorn  6.108534
 2  RandomForestRegressor  hyperimpute  7.385313
 3  RandomForestRegressor         MIDA  7.576606
 4  RandomForestRegressor           lm  8.360542,
 'RRMSE_kom':                    model      method  RRMSE_kom
 0  RandomForestRegressor    sinkhorn   0.803264
 1  RandomForestRegressor  missforest   0.934597
 2   MultiOutputRegressor    sinkhorn   0.975357
 3   MultiOutputRegressor    sinkhorn   0.975357
 4                  Ridge    sinkhorn   0.991152}

In [61]:
type(top5)

dict

In [64]:
top5.get('R2_kom')

,model,method,R2_kom
0,RandomForestRegressor,sinkhorn,0.354767
1,RandomForestRegressor,missforest,0.126529
2,MultiOutputRegressor,sinkhorn,0.048679
3,MultiOutputRegressor,sinkhorn,0.048679
4,Ridge,sinkhorn,0.017618


In [65]:
top5.get('RMSE_kom')

,model,method,RMSE_kom
0,RandomForestRegressor,missforest,6.055875
1,RandomForestRegressor,sinkhorn,6.108534
2,RandomForestRegressor,hyperimpute,7.385313
3,RandomForestRegressor,MIDA,7.576606
4,RandomForestRegressor,lm,8.360542


In [66]:
top5.get('RRMSE_kom')

,model,method,RRMSE_kom
0,RandomForestRegressor,sinkhorn,0.803264
1,RandomForestRegressor,missforest,0.934597
2,MultiOutputRegressor,sinkhorn,0.975357
3,MultiOutputRegressor,sinkhorn,0.975357
4,Ridge,sinkhorn,0.991152


In [53]:
result = pd.DataFrame(result)
result.to_csv('./uhpc_result.csv')